# <div align="center" style="color: #ff5733;">Flag Creation</div>

# Declare Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None) 

In [78]:
sq = """
with mau4m as 
(select customer_id, registration_date from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data
where 
    tx_cnt_total_transactions_bw_3m_and_4m >=1
    and 
    tx_cnt_active_loans_end_4m = 0
),
mau5m as
(select customer_id, registration_date from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data
where 
    tx_cnt_total_transactions_bw_4m_and_5m >=1
    and
    tx_cnt_active_loans_end_5m = 0
),
lmt as 
(select customerId, startApplyDateTime, approvedDateTime from  `risk_credit_mis.loan_master_table`)
select 
ctd.customer_id,
ctd.registration_date,
ctd.tx_first_product,
case when ctd.customer_id in (select customer_id from mau4m) then 1 else 0 end tx_mau4m_active,
case when ctd.customer_id in (select customer_id from mau5m) then 1 else 0 end tx_mau5m_active,

---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay', 'Unsecured Loan')
   then case when ctd.customer_id in (select customer_id from mau4m)
    then 0
    else 1 
end end tx_Churn_flag_4m,

---If the user is also Active (as per MAU definition) as of 150th Day from the Onboarding date then 0, otherwise 1

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay', 'Unsecured Loan') then 
   case when ctd.customer_id in (select customer_id from mau5m)
    then 0
    else 1
end end tx_Churn_flag_5m,

---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date, AND did not submit any loan application to Tonik

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay') then 
    case when ctd.customer_id in (select customer_id from mau4m)                                                                --- User is active MAU as of 120 day from Onboarding date
    and (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between date(ctd.registration_date)
                                                               and date_add(date(ctd.registration_date), interval 120 day)) = 0
    then 1
    else 0
end end tx_loanApplicationflag_4m,

case when (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between date(ctd.registration_date) 
                                                               and date_add(date(ctd.registration_date), interval 120 day)) = 0 then 1 else 0 end submittedappliction4m,
                                                               
---If the users submitted any Cash loan application within 5th month (next 30 days from the 120th day from onboarding date) at Tonik then 1, otherwise 0

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay') then
    case when ctd.customer_id in (select customer_id from mau5m)
    and(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)) = 0
    then 1
    else 0
end end tx_loanApplicationflag_5m,

case when (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)) = 0 then 1 else 0 end submittedappliction5m,
                                                               
---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date, AND did not submit any loan application to Tonik within 120 days from onboarding date       
                                                        
case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay') then
    case when ctd.customer_id in (select customer_id from mau5m)
    and(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)
                                                               and date(approvedDateTime) is not null 
                                                               and flagApproval = 1
                                                               and new_loan_type in ('Quick', 'Flex')) = 0
    then 1
    else 0
end end tx_loanApprovedflag_5m,
---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date AND never NPL AND current DPD status <=FSPD30 as of 120th day from the Onboarding date
case when ctd.tx_first_product in ('Unsecured Loan') then
     case when ctd.customer_id in (select customer_id from mau4m)
     and (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                                            and Max_Ever_DPD >= 180
                                                                            and new_loan_type in ('Quick', 'Flex')) = 0
     and (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
                   inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
                   inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
                   where lmt.customerId = cast(ctd.customer_id as numeric)
                   and lbfrc.bucketDate = date_add(ctd.registration_date, interval 120 day)
                   and ldd.min_inst_def30 in (1, 2)
                   and lmt.new_loan_type in ('Quick', 'Flex')
                  ) = 0
     then 0
     else 1
end end tx_FSPD30_4m,
(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                                            and Max_Ever_DPD >= 180
                                                                            and new_loan_type in ('Quick', 'Flex')) npl,
---If the user becomes FSPD30 within next 90 days from the observation date (120th day from the onboarding date) then 1; otherwise 0
case when ctd.tx_first_product in ('Unsecured Loan') then
     case when ctd.customer_id in (select customer_id from mau5m)
     and (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
                   inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
                   inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
                   where lmt.customerId = cast(ctd.customer_id as numeric)
                   and lbfrc.bucketDate = date_add(ctd.registration_date, interval 150 day)
                   and ldd.min_inst_def30 in (1, 2)
                   and lmt.new_loan_type in ('Quick', 'Flex')
                  ) = 0
     then 0
     else 1
end end tx_FSPD30_5m,
(select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
                   inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
                   inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
                   where lmt.customerId = cast(ctd.customer_id as numeric)
                   and lbfrc.bucketDate = date_add(ctd.registration_date, interval 120 day)
                   and ldd.min_inst_def30 in (1, 2)
                   and lmt.new_loan_type in ('Quick', 'Flex')
                  ) fspd304m,
(select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
                   inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
                   inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
                   where lmt.customerId = cast(ctd.customer_id as numeric)
                   and lbfrc.bucketDate = date_add(ctd.registration_date, interval 150 day)
                   and ldd.min_inst_def30 in (1, 2)
                   and lmt.new_loan_type in ('Quick', 'Flex')
                  ) fspd305m
from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data ctd
"""

df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')



Job ID d6d77c78-de3c-4cfe-8525-f4f34db3d1b1 successfully executed: |          |    
Downloading: 100%|██████████|


In [79]:
df.shape

(473102, 17)

In [81]:
df[(df['tx_Churn_flag_4m']==1) & (df['submittedappliction4m']== 1)]
# df[df['submittedappliction']== 0]

,customer_id,registration_date,tx_first_product,tx_mau4m_active,tx_mau5m_active,tx_Churn_flag_4m,tx_Churn_flag_5m,tx_loanApplicationflag_4m,submittedappliction4m,tx_loanApplicationflag_5m,submittedappliction5m,tx_loanApprovedflag_5m,tx_FSPD30_4m,npl,tx_FSPD30_5m,fspd304m,fspd305m
6,2416158,2024-03-01,Individual Stash,0,1,1,0,0,1,1,1,1,<NA>,0,<NA>,0,0
15,2287299,2023-10-28,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
94,2092381,2023-06-13,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
221,2431808,2024-03-16,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
248,1853738,2023-01-07,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472729,1875339,2023-01-26,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
472742,2155680,2023-07-25,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
473010,2066681,2023-05-29,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0
473072,2044182,2023-05-14,Individual Stash,0,0,1,1,0,1,0,1,0,<NA>,0,<NA>,0,0


In [82]:
sq = """select customerId, digitalLoanAccountId, startApplyDateTime, startInitiateDateTime from `risk_credit_mis.loan_master_table` where customerId = 2001751;"""
check = client.query(sq).to_dataframe()
check

,customerId,digitalLoanAccountId,startApplyDateTime,startInitiateDateTime
0,2001751,8357b50a-a6a5-4d7b-ae14-76fd4271bd23,NaT,2023-04-18 20:45:31


In [83]:
df.sample(20)

,customer_id,registration_date,tx_first_product,tx_mau4m_active,tx_mau5m_active,tx_Churn_flag_4m,tx_Churn_flag_5m,tx_loanApplicationflag_4m,submittedappliction4m,tx_loanApplicationflag_5m,submittedappliction5m,tx_loanApprovedflag_5m,tx_FSPD30_4m,npl,tx_FSPD30_5m,fspd304m,fspd305m
333655,1961135,2023-03-26,Unknown,0,0,<NA>,<NA>,<NA>,0,<NA>,1,<NA>,<NA>,0,<NA>,0,0
116461,2103175,2023-06-21,Unknown,0,0,<NA>,<NA>,<NA>,0,<NA>,1,<NA>,<NA>,0,<NA>,0,0
247860,2097714,2023-06-17,Unknown,0,0,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0,<NA>,0,0
256676,2048116,2023-05-17,Unknown,0,0,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0,<NA>,0,0
224338,2421438,2024-03-06,Unknown,0,0,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0,<NA>,0,0
258114,1902687,2023-02-16,Unknown,0,0,<NA>,<NA>,<NA>,0,<NA>,1,<NA>,<NA>,0,<NA>,0,0
354318,2106778,2023-06-24,Unknown,0,0,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0,<NA>,0,0
378409,1855800,2023-01-09,Unknown,0,0,<NA>,<NA>,<NA>,0,<NA>,1,<NA>,<NA>,0,<NA>,0,0
71965,2377956,2024-01-23,Unknown,0,0,<NA>,<NA>,<NA>,0,<NA>,1,<NA>,<NA>,0,<NA>,0,0
246004,2120471,2023-07-05,TSA Top-Up,0,0,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0,<NA>,0,0


In [84]:
df.to_csv("Flag_table.csv", index = False)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473102 entries, 0 to 473101
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                473102 non-null  object
 1   registration_date          473102 non-null  dbdate
 2   tx_first_product           473102 non-null  object
 3   tx_mau4m_active            473102 non-null  Int64 
 4   tx_mau5m_active            473102 non-null  Int64 
 5   tx_Churn_flag_4m           38558 non-null   Int64 
 6   tx_Churn_flag_5m           38558 non-null   Int64 
 7   tx_loanApplicationflag_4m  24847 non-null   Int64 
 8   submittedappliction4m      473102 non-null  Int64 
 9   tx_loanApplicationflag_5m  24847 non-null   Int64 
 10  submittedappliction5m      473102 non-null  Int64 
 11  tx_loanApprovedflag_5m     24847 non-null   Int64 
 12  tx_FSPD30_4m               13711 non-null   Int64 
 13  npl                        473102 non-null  

In [86]:
df.columns

Index(['customer_id', 'registration_date', 'tx_first_product',
       'tx_mau4m_active', 'tx_mau5m_active', 'tx_Churn_flag_4m',
       'tx_Churn_flag_5m', 'tx_loanApplicationflag_4m',
       'submittedappliction4m', 'tx_loanApplicationflag_5m',
       'submittedappliction5m', 'tx_loanApprovedflag_5m', 'tx_FSPD30_4m',
       'npl', 'tx_FSPD30_5m', 'fspd304m', 'fspd305m'],
      dtype='object')

In [42]:
sq = """drop table if exists prj-prod-dataplatform.worktable_data_analysis.customer_360_flags;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=32ce1b88-e920-4e5b-b926-13ec68738dab>

In [43]:
# Define the dataset and table name
dataset_id = 'worktable_data_analysis'
table_id = 'customer_360_flags'

# Define the schema based on the DataFrame structure
schema = [
    bigquery.SchemaField("customer_id", "STRING"),
    bigquery.SchemaField("registration_date", "TIMESTAMP"),  # Update to TIMESTAMP
    bigquery.SchemaField("tx_first_product", "STRING"),
    bigquery.SchemaField("tx_mau4m_active", "INT64"),
    bigquery.SchemaField("tx_mau5m_active", "INT64"),
    bigquery.SchemaField("tx_Churn_flag_4m", "INT64"),
    bigquery.SchemaField("tx_Churn_flag_5m", "INT64"),
    bigquery.SchemaField("tx_loanApplicationflag_4m", "INT64"),
    bigquery.SchemaField("submittedappliction4m", "INT64"),
    bigquery.SchemaField("tx_loanApplicationflag_5m", "INT64"),
    bigquery.SchemaField("submittedappliction5m", "INT64"),
    bigquery.SchemaField("tx_loanApprovedflag_5m", "INT64"),
    bigquery.SchemaField("tx_FSPD30_4m", "INT64"),
    bigquery.SchemaField("tx_FSPD30_5m", "INT64"),
    bigquery.SchemaField("fspd304m", "INT64"),
    bigquery.SchemaField("fspd305m", "INT64")
    
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table customer_360_flags created in dataset worktable_data_analysis.


In [87]:
df[['tx_mau4m_active', 'tx_mau5m_active', 'tx_Churn_flag_4m', 'tx_Churn_flag_5m', 'tx_loanApplicationflag_4m', 
                  'submittedappliction4m', 'tx_loanApplicationflag_5m','submittedappliction5m', 'tx_loanApprovedflag_5m',
                  'tx_FSPD30_4m','tx_FSPD30_5m']].value_counts(dropna=False)

tx_mau4m_active  tx_mau5m_active  tx_Churn_flag_4m  tx_Churn_flag_5m  tx_loanApplicationflag_4m  submittedappliction4m  tx_loanApplicationflag_5m  submittedappliction5m  tx_loanApprovedflag_5m  tx_FSPD30_4m  tx_FSPD30_5m
0                0                <NA>              <NA>              <NA>                       0                      <NA>                       1                      <NA>                    <NA>          <NA>            258065
                                                                                                 1                      <NA>                       1                      <NA>                    <NA>          <NA>            162426
                                  1                 1                 0                          0                      0                          1                      0                       <NA>          <NA>             12699
                                                                      <NA>            

In [88]:
df1 = df.copy()

In [89]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473102 entries, 0 to 473101
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                473102 non-null  object
 1   registration_date          473102 non-null  dbdate
 2   tx_first_product           473102 non-null  object
 3   tx_mau4m_active            473102 non-null  Int64 
 4   tx_mau5m_active            473102 non-null  Int64 
 5   tx_Churn_flag_4m           38558 non-null   Int64 
 6   tx_Churn_flag_5m           38558 non-null   Int64 
 7   tx_loanApplicationflag_4m  24847 non-null   Int64 
 8   submittedappliction4m      473102 non-null  Int64 
 9   tx_loanApplicationflag_5m  24847 non-null   Int64 
 10  submittedappliction5m      473102 non-null  Int64 
 11  tx_loanApprovedflag_5m     24847 non-null   Int64 
 12  tx_FSPD30_4m               13711 non-null   Int64 
 13  npl                        473102 non-null  

In [90]:
# Identify the column types
date_columns = df1.select_dtypes(include=['datetime', 'dbdate']).columns
numeric_columns = df1.select_dtypes(include=['number', 'Int64']).columns
object_columns = df1.select_dtypes(include=['object']).columns

# Fill missing values for numeric columns with -1
df1[numeric_columns] = df1[numeric_columns].fillna(-1)

# Fill missing values for datetime columns with a specific date
df1[date_columns] = df1[date_columns].fillna(pd.Timestamp("1900-01-01"))

# Fill missing values for object columns with a placeholder (if needed)
df1[object_columns] = df1[object_columns].fillna("Unknown")

# Check if fillna was successful
print(df1.isna().sum())

customer_id                  0
registration_date            0
tx_first_product             0
tx_mau4m_active              0
tx_mau5m_active              0
tx_Churn_flag_4m             0
tx_Churn_flag_5m             0
tx_loanApplicationflag_4m    0
submittedappliction4m        0
tx_loanApplicationflag_5m    0
submittedappliction5m        0
tx_loanApprovedflag_5m       0
tx_FSPD30_4m                 0
npl                          0
tx_FSPD30_5m                 0
fspd304m                     0
fspd305m                     0
dtype: int64


In [91]:
dfd = df1.groupby(['tx_mau4m_active', 'tx_mau5m_active', 'tx_Churn_flag_4m', 'tx_Churn_flag_5m', 'tx_loanApplicationflag_4m', 
                  'submittedappliction4m', 'tx_loanApplicationflag_5m','submittedappliction5m', 'tx_loanApprovedflag_5m',
                  'tx_FSPD30_4m','tx_FSPD30_5m']).agg(cntcustomer=('customer_id', 'nunique')).reset_index()
dfd

,tx_mau4m_active,tx_mau5m_active,tx_Churn_flag_4m,tx_Churn_flag_5m,tx_loanApplicationflag_4m,submittedappliction4m,tx_loanApplicationflag_5m,submittedappliction5m,tx_loanApprovedflag_5m,tx_FSPD30_4m,tx_FSPD30_5m,cntcustomer
0,0,0,-1,-1,-1,0,-1,0,-1,-1,-1,4649
1,0,0,-1,-1,-1,0,-1,1,-1,-1,-1,258065
2,0,0,-1,-1,-1,1,-1,0,-1,-1,-1,1267
3,0,0,-1,-1,-1,1,-1,1,-1,-1,-1,162426
4,0,0,1,1,-1,0,-1,0,-1,1,1,266
5,0,0,1,1,-1,0,-1,1,-1,1,1,12380
6,0,0,1,1,-1,1,-1,0,-1,1,1,57
7,0,0,1,1,-1,1,-1,1,-1,1,1,138
8,0,0,1,1,0,0,0,0,0,-1,-1,516
9,0,0,1,1,0,0,0,1,0,-1,-1,12699


In [92]:
# Define the list of values to filter
a = ['Group Stash', 'Individual Stash']

# Filter the DataFrame for rows where 'tx_first_product' is in the list 'a'
df_filtered = df1[df1['tx_first_product'].isin(a)]

# Group by the desired columns and aggregate with the correct 'nunique'
df_grouped = df_filtered.groupby(['tx_first_product', 'tx_mau4m_active', 'submittedappliction4m', 'tx_loanApplicationflag_4m', 'tx_loanApplicationflag_5m']).agg(
    customer_unique_count=('customer_id', 'nunique')
).reset_index()

df_grouped.sort_values(by = 'tx_mau4m_active')

,tx_first_product,tx_mau4m_active,submittedappliction4m,tx_loanApplicationflag_4m,tx_loanApplicationflag_5m,customer_unique_count
0,Group Stash,0,0,0,0,842
1,Group Stash,0,0,0,1,9
2,Group Stash,0,1,0,0,669
3,Group Stash,0,1,0,1,98
11,Individual Stash,0,1,0,1,480
10,Individual Stash,0,1,0,0,7832
9,Individual Stash,0,0,0,1,78
8,Individual Stash,0,0,0,0,12391
7,Group Stash,1,1,1,1,226
6,Group Stash,1,1,1,0,54


: 

In [68]:
sq = """
with mau4m as 
(select customer_id, registration_date from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data
where 
    tx_cnt_total_transactions_bw_3m_and_4m >=1
    and 
    tx_cnt_active_loans_end_4m >=1
)
select * from mau4m;
"""

mau4m = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID a2aa05b9-2e64-4390-bb77-fd04bfc67aa5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [69]:
mau4m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40691 entries, 0 to 40690
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        40691 non-null  object
 1   registration_date  40691 non-null  dbdate
dtypes: dbdate(1), object(1)
memory usage: 635.9+ KB


In [70]:
sq = """
select * from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data where tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay');
"""

tfp = client.query(sq).to_dataframe()


In [71]:
sq = """select customerId, startApplyDateTime from `risk_credit_mis.loan_master_table`;"""

lmt = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 39196a85-3873-427e-a040-e385929e199f successfully executed: 100%|██████████|
Query is running:   0%|          |
Downloading: 100%|██████████|


In [72]:
lmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718639 entries, 0 to 1718638
Data columns (total 2 columns):
 #   Column              Dtype         
---  ------              -----         
 0   customerId          Int64         
 1   startApplyDateTime  datetime64[us]
dtypes: Int64(1), datetime64[us](1)
memory usage: 27.9 MB


In [74]:
mergeddf = tfp[['customer_id', 'tx_first_product']].merge(mau4m, left_on = 'customer_id', right_on='customer_id', how = 'inner')
mergeddf.shape

(394, 3)

In [75]:
mergeddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        394 non-null    object
 1   tx_first_product   394 non-null    object
 2   registration_date  394 non-null    dbdate
dtypes: dbdate(1), object(2)
memory usage: 9.4+ KB


In [76]:

# Convert 'registration_date' to datetime if it's not already in datetime format
mergeddf['registration_date'] = pd.to_datetime(mergeddf['registration_date'])

# Convert 'customerId' to string to match the format of 'customer_id' for merging
lmt['customerId'] = lmt['customerId'].astype(str)

# Merge the DataFrames on customer_id (from mergeddf) and customerId (from lmt)
merged_data = pd.merge(mergeddf, lmt, left_on='customer_id', right_on='customerId', how='inner')

# Filter rows where 'startApplyDateTime' is between 'registration_date' and 'registration_date + 120 days'
filtered_data = merged_data[
    (merged_data['startApplyDateTime'] >= merged_data['registration_date']) &
    (merged_data['startApplyDateTime'] <= merged_data['registration_date'] + pd.Timedelta(days=120))
]

# Display the resulting filtered DataFrame



In [77]:
filtered_data.groupby('tx_first_product')['customer_id'].nunique()

tx_first_product
Bills Pay            11
Group Stash          21
Individual Stash    362
Name: customer_id, dtype: int64